In [174]:
import threading

In [356]:
def task():
    print("hi from task, thread:", threading.get_native_id())

t = threading.Thread(target=task)
t.start()
print("hi from main, thread:", threading.get_native_id()) # thread ID

hi from task, thread:hi from main, thread: 264
 1185


In [378]:
total = 0
def task2(count):
    global total
    for i in range(count):
        total += 1

t = threading.Thread(target=task2, args=[1_000])
t.start()
t.join() # main thread will block, until t is done
total

1000

In [384]:
total = 0
def task3(count):
    global total
    for i in range(count):
        total += 1

t1 = threading.Thread(target=task3, args=[1_000_000])
t2 = threading.Thread(target=task3, args=[1_000_000])
t1.start()
t2.start()
t1.join() # main thread will block, until t is done
t2.join()
total

1103104

In [386]:
import dis
dis.dis("total += 1")

  0           RESUME                   0

  1           LOAD_NAME                0 (total)
              LOAD_CONST               0 (1)
              BINARY_OP               13 (+=)
              STORE_NAME               0 (total)
              RETURN_CONST             1 (None)
